## PROJE
##### https://www.kaggle.com/deepmatrix/imdb-5000-movie-dataset 
##### Konu: imdb veri setini pyspark ile analiz etmek.

In [2]:
import findspark 
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
findspark.init()
findspark.find()

'/opt/conda/lib/python3.10/site-packages/pyspark'

In [3]:
try:
    sc.stop()
except NameError:
    sc = SparkContext()


23/04/14 19:10:11 WARN Utils: Your hostname, codespaces-282530 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/04/14 19:10:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/14 19:10:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
rdd = sc.textFile('/workspaces/codespaces-jupyter/data/movie_metadata.csv')

In [5]:
rdd.map(lambda x: len(x.split(','))).map(lambda x:(x,1)).countByKey()

defaultdict(int, {28: 4975, 29: 61, 30: 6, 31: 2})

In [6]:
spark = SparkSession.builder.appName("Film Analizi").getOrCreate()
films = spark.read.options(header="true", inferSchema="true") \
             .csv("/workspaces/codespaces-jupyter/data/movie_metadata.csv")

1- Hangi ülkenin kaç tane filmi ilk 5 bin arasında yer alıyor?


In [7]:
# read fonksiyonunu kullanarak CSV dosyasını yükleyelim
# `header="true"` seçeneğiyle başlık satırını atlayalım
# `inferSchema="true"` seçeneğiyle şemayı otomatik olarak çıkaralım
films = spark.read.options(header="true", inferSchema="true") \
             .csv("/workspaces/codespaces-jupyter/data/movie_metadata.csv")

# `groupBy` fonksiyonuyla "country" sütununa göre gruplandırma yapalım
# `count` fonksiyonuyla her ülkenin film sayısını hesaplayalım
# `sort` fonksiyonuyla "count" sütununa göre büyükten küçüğe sıralama yapalım
# `limit` fonksiyonuyla ilk 5000 kaydı seçelim
country_counts = (films.groupBy("country")
                       .count()
                       .sort("count", ascending=False)
                       .limit(5000))

# `show` fonksiyonuyla sonucu ekrana yazdıralım
country_counts.show(5) # kaç ülke istersek o kadar yazarız

+-------+-----+
|country|count|
+-------+-----+
|    USA| 3807|
|     UK|  448|
| France|  154|
| Canada|  126|
|Germany|   97|
+-------+-----+
only showing top 5 rows



2- Her ülkenin ortalama imdb puanı kaç?


In [8]:
# 1.yol

from pyspark.sql.functions import avg

country_avg = (films.groupBy("country")
                    .agg(avg("imdb_score").alias("avg_score"))
                    .sort("avg_score", reversed=False)
                    .limit(115)
)
country_avg.show(100)

+--------------------+------------------+
|             country|         avg_score|
+--------------------+------------------+
|             Bahamas|               4.4|
|            New Line|               4.4|
|               Aruba|               4.8|
|                Peru|               5.4|
|            Cambodia|               5.6|
|             Belgium|               5.6|
|             Nigeria|               5.6|
|             Georgia|               5.6|
|         Switzerland|5.8999999999999995|
|              Turkey|               6.0|
|            Slovakia|               6.0|
|            Thailand|              6.08|
|              Russia| 6.081818181818182|
|            Bulgaria|               6.1|
|              Canada| 6.161904761904762|
|         South Korea| 6.257142857142858|
|         Philippines|               6.3|
|       Official site|               6.3|
|             Germany| 6.340206185567008|
|                 USA| 6.367428421329131|
|            Slovenia|            

In [20]:
# 2.yol 
country_groups = films.groupBy("country")
country_groups_avg = (country_groups
                      .agg(avg("imdb_score").alias("imdb avarage"))
                      .sort("imdb avarage", ascending=False)
                      .show(20))


+--------------------+------------------+
|             country|      imdb avarage|
+--------------------+------------------+
|          Kyrgyzstan|               8.7|
|               Libya|               8.4|
|United Arab Emirates|               8.2|
|        Soviet Union|               8.1|
|               Egypt|               8.1|
|                Iran|             7.725|
|              Poland|              7.62|
|           Indonesia|               7.6|
|              Israel|7.5249999999999995|
|              Sweden| 7.516666666666666|
|            Colombia|               7.5|
|           Argentina|               7.5|
|            Cameroon|               7.5|
|               Kenya|               7.4|
|         Afghanistan|               7.4|
|             Iceland| 7.333333333333333|
|         New Zealand| 7.279999999999999|
|              Brazil|7.2749999999999995|
|        West Germany| 7.266666666666667|
|             Finland|               7.2|
+--------------------+------------

In [10]:
# eğer virgülden sonrasını yuvarlamak istersek aynı koda şunu eklememiz yeterlidir

from pyspark.sql.functions import round 

country_groups_avg_but_Rounded = (country_groups
                                  .agg(round(avg("imdb_score"),1).alias("imdb avarage"))
                                  .sort("imdb avarage", ascending=False)
                                  .show(10))


+--------------------+------------+
|             country|imdb avarage|
+--------------------+------------+
|          Kyrgyzstan|         8.7|
|               Libya|         8.4|
|United Arab Emirates|         8.2|
|        Soviet Union|         8.1|
|               Egypt|         8.1|
|                Iran|         7.7|
|              Poland|         7.6|
|           Indonesia|         7.6|
|              Israel|         7.5|
|              Sweden|         7.5|
+--------------------+------------+
only showing top 10 rows



3- Her yönetmenin ortalama imdb puanı kaç?


In [11]:
director_groups = films.groupBy("director_name")

director_avg_scores = (director_groups
                       .agg(round(avg("imdb_score"),1)
                            .alias("avg_score")))
director_avg_scores_sorted = director_avg_scores.sort("avg_score", ascending=False).show(15)

+--------------------+---------+
|       director_name|avg_score|
+--------------------+---------+
|      John Blanchard|      9.5|
|    Mitchell Altieri|      8.7|
|           Cary Bell|      8.7|
|    Sadyk Sher-Niyaz|      8.7|
|        Mike Mayhall|      8.6|
|     Charles Chaplin|      8.6|
|     Damien Chazelle|      8.5|
|          Ron Fricke|      8.5|
|        Majid Majidi|      8.5|
|          Raja Menon|      8.5|
|        Sergio Leone|      8.5|
|      S.S. Rajamouli|      8.4|
|Rakeysh Omprakash...|      8.4|
|     Catherine Owens|      8.4|
|           Jay Oliva|      8.4|
+--------------------+---------+
only showing top 15 rows



4- En çok hangi türde film ilk 5000'de yer almıştır


In [12]:
# 1.Yol

from pyspark.sql.functions import desc

# Sütunları seçelim
movies = films.select("genres", "imdb_score")

# sadece ilk 5000 filmden oluşan bir veri kümesi oluşturalım
movies = movies.orderBy(desc("imdb_score")).limit(5000)

# '|' karakteriyle ayrılmış türlerin listesini alalım
genres = movies.rdd.flatMap(lambda row: row[0].split("|"))

# her türün sayısını say ve en yüksek olanı alalım
top_genre = genres.countByValue().items()
top_genre = sorted(top_genre, key=lambda x: x[1], reverse=True)[0][0]

# cevabı yazdıralım
print("En çok {} türündeki filmler ilk 5 binde yer almaktadır.".format(top_genre))

En çok Drama türündeki filmler ilk 5 binde yer almaktadır.


In [13]:
# 2.Yol

from pyspark.sql.functions import desc, split

movies_filtered = films.filter(films.gross.isNotNull())

movies = movies_filtered.select("genres", "imdb_score")

top_movies = movies.orderBy(desc("imdb_score")).limit(5000)

top_genres = top_movies.select(split(top_movies.genres, "\|").alias("genre")).selectExpr("explode(genre) as genre")

top_genres.groupBy("genre").count().sort(desc("count")).show(1, False)

+-----+-----+
|genre|count|
+-----+-----+
|Drama|2140 |
+-----+-----+
only showing top 1 row



In [19]:
# 3.Yol

from pyspark.sql.functions import explode, split

df_films = films.withColumn("genre", explode(split("genres","\|")))
category_count = (df_films
                  .withColumn("genre", 
                              explode(split("genres", "\|")))
                  .groupBy("genre").count())
category_count = category_count.sort("count",ascending=False)
category_count.show(10) # bizim ihtiyacımız olan en çok hangi tür olduğudur
                        # bu yüzden show içerisine kaç yazdığımızın bir önemi yok

+---------+-----+
|    genre|count|
+---------+-----+
|    Drama| 7706|
|   Comedy| 5485|
| Thriller| 4865|
|   Action| 4253|
|Adventure| 3622|
|  Romance| 3516|
|    Crime| 3063|
|  Fantasy| 2377|
|   Sci-Fi| 2290|
|   Family| 2204|
+---------+-----+
only showing top 10 rows



5- En fazla bütçeye sahip olan filmler toplamda ne kadar hasılat elde etti? En fazla bütçeye sahip olan ilk 20 filmi karşılaştırın ortalama hasılatı yazdırın.


In [14]:
from pyspark.sql.functions import desc

# filmleri seçin ve bütçeye göre sıralayalım
budget_revenue = films.select('budget', 'gross', 'movie_title').orderBy(desc('budget')).head(20)

# toplam hasılatı hesaplayalım
total_revenue = sum([film.gross for film in budget_revenue])

# ortalama hasılatı hesaplayalım
average_revenue = total_revenue/20

# cevabı yazdıralım
print("En yüksek bütçeye sahip olan ilk 20 film, toplamda {} hasılat elde etti. Ortalama hasılat ise {}.".format(total_revenue, average_revenue))


En yüksek bütçeye sahip olan ilk 20 film, toplamda 1549112804 hasılat elde etti. Ortalama hasılat ise 77455640.2.
